In [2]:
import os
import pandas as pd
import numpy as np

In [4]:
startdate = "2022-03-01"
enddate = "2022-03-31"

files = os.listdir("validateddata")
files.sort()
dates = [file.split(".")[0] for file in files]

filtered_dates = []

for date in dates:
    if (date >= startdate) and (date <= enddate):
        filtered_dates.append(date)

In [7]:
df_concat = pd.DataFrame([])
for date in filtered_dates:
    df = pd.read_csv(f"validateddata/{date}.csv", header=0, parse_dates=["timestamp"], index_col=0)
    df_concat = pd.concat([df_concat, df])

df_resampled = df_concat.resample("10T", label="right").mean()

In [11]:
df_feature = df_resampled.loc[:, ["SAlNextHr", "NormalizedEeg", "Temperature"]]
df_label = df_resampled.loc[:, ["Eeg", "Eed"]]
df_label.rename(columns={"Eeg": "Eeg_label", "Eed": "Eed_label"}, inplace=True)
df_label = df_label.iloc[6:].reset_index(drop=True)
df_label.index = df_resampled.index[:-6]
df_dataset = pd.concat([df_feature, df_label], axis=1)
df_dataset.dropna(inplace=True)
df_dataset.to_csv(f"dataset/ann_{startdate}_{enddate}.csv", index=True, header=True)